# Business Problem and Background

    I represent a national donut shop, Galaxy Donuts, and we are expanding into the state of Minnesota. Our focus is on finding locations in the city of Minneapolis that will maximize profitability for the company. The biggest problem identified by our new CEO is our donut competition has been established for some time in the Minneapolis area. However, based on our proprietary Donut Density Ratio (DDR) we know there is room for at least one location for our brand, if not more. 
    
    We will need to analyze the 83 neighborhoods of Minneapolis proper, and make the decision based on optimal distance from location of competitor donut shops. Finally, we want to place the location of our stores in higher density areas that will ensure plenty of traffic, as well as other activities for our customers to do after they have their minds blown by our donuts.


# The Data

    I will be using the Foursquare API to identify all donut shops in and around the city of Minneapolis. These will be indicated with red dots on the map. Next, I will determine which neighborhoods are the optimal/maximal distance from the other donut shops.
    
    Once I have that list of neighborhoods, I will analyze those specific neighborhoods, dividing out all residential and industrial space, leaving only commercial space. If needed I will utilize city of Minneapolis open source data to help with this determination. Then I will use the Foursquare API to find businesses that might complement a donut shop (list still being determined, but will be included in the final report), and attempt to place the neighborhood in an area where the mean customer scores are highest, indicating a more satisfying experience/location. 
    
    The final work product will be a map of Minneapolis, linked to the Foursquare API denoting all competitors, and I will use all the data discussed above to place the coveted blue dot where Galaxy Donuts will open its newest location(s). All the findings will be returned in a formatted report with visuals, and I will also write the computer program used for analysis in a Jupyter Notebook and share it on my Github page with my colleagues.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [2]:
CLIENT_ID = 'G5T2H5YFJAPACKLEQST13SJLTRRB4QMHFGJMDF0ZSRHSXPDX' # your Foursquare ID
CLIENT_SECRET = 'WB3CGZOIHBTSLJYERUI3VORHYWDEAF5R034JZRT3M4TVXRYC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: G5T2H5YFJAPACKLEQST13SJLTRRB4QMHFGJMDF0ZSRHSXPDX
CLIENT_SECRET:WB3CGZOIHBTSLJYERUI3VORHYWDEAF5R034JZRT3M4TVXRYC


In [7]:
address = 'Minneapolis, MN'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

44.9706705 -93.268217735811


In [8]:
radius = 30000
print(' .... OK!')

 .... OK!


In [15]:
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d148941735&intent=browse&radius=30000&client_id={}&client_secret={}&ll={},{}&v={}&query='.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d148941735&intent=browse&radius=30000&client_id=G5T2H5YFJAPACKLEQST13SJLTRRB4QMHFGJMDF0ZSRHSXPDX&client_secret=WB3CGZOIHBTSLJYERUI3VORHYWDEAF5R034JZRT3M4TVXRYC&ll=44.9706705,-93.268217735811&v=20180604&query='

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da75998f9dbde003843ccf5'},
 'response': {'venues': [{'id': '584c04838ee56059df135672',
    'name': "Dunkin'",
    'location': {'address': 'MSP Airport, Terminal 1',
     'crossStreet': 'Main Concourse',
     'lat': 44.88374364677225,
     'lng': -93.21137493853037,
     'labeledLatLngs': [{'label': 'display',
       'lat': 44.88374364677225,
       'lng': -93.21137493853037}],
     'distance': 10663,
     'postalCode': '55111',
     'cc': 'US',
     'city': 'Saint Paul',
     'state': 'MN',
     'country': 'United States',
     'formattedAddress': ['MSP Airport, Terminal 1 (Main Concourse)',
      'Saint Paul, MN 55111',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d148941735',
      'name': 'Donut Shop',
      'pluralName': 'Donut Shops',
      'shortName': 'Donuts',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/donuts_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1571248536

In [17]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,584c04838ee56059df135672,Dunkin',"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",v-1571248536,False,"MSP Airport, Terminal 1",Main Concourse,44.883744,-93.211375,"[{'label': 'display', 'lat': 44.88374364677225...",10663,55111,US,Saint Paul,MN,United States,"[MSP Airport, Terminal 1 (Main Concourse), Sai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,580fa20238facc4e6ffd1569,Angel Food Bakery & Donut Bar,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1571248536,False,Concourse E,NaN,44.885040,-93.212566,"[{'label': 'display', 'lat': 44.8850400628044,...",10492,55111,US,Saint Paul,MN,United States,"[Concourse E, Saint Paul, MN 55111, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b5864b8f964a520c35528e3,Donut Connection,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",v-1571248536,False,1037 1st Ave E,NaN,44.800545,-93.513115,"[{'label': 'display', 'lat': 44.80054540165129...",27050,55379,US,Shakopee,MN,United States,"[1037 1st Ave E, Shakopee, MN 55379, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58517cc7739d855b44e53255,Dunkin',"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",v-1571248536,False,2425 Rice St,NaN,45.013967,-93.106732,"[{'label': 'display', 'lat': 45.01396688648932...",13595,55113,US,Roseville,MN,United States,"[2425 Rice St, Roseville, MN 55113, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51254ee3e4b0597635a30975,Glam Doll Donuts,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",v-1571248536,False,2605 Nicollet Ave,at E 26th St.,44.955156,-93.277816,"[{'label': 'display', 'lat': 44.95515607246256...",1885,55408,US,Minneapolis,MN,United States,"[2605 Nicollet Ave (at E 26th St.), Minneapoli...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Dunkin',Donut Shop,"MSP Airport, Terminal 1",Main Concourse,44.883744,-93.211375,"[{'label': 'display', 'lat': 44.88374364677225...",10663,55111,US,Saint Paul,MN,United States,"[MSP Airport, Terminal 1 (Main Concourse), Sai...",NaN,584c04838ee56059df135672
1,Angel Food Bakery & Donut Bar,Bakery,Concourse E,NaN,44.885040,-93.212566,"[{'label': 'display', 'lat': 44.8850400628044,...",10492,55111,US,Saint Paul,MN,United States,"[Concourse E, Saint Paul, MN 55111, United Sta...",NaN,580fa20238facc4e6ffd1569
2,Donut Connection,Donut Shop,1037 1st Ave E,NaN,44.800545,-93.513115,"[{'label': 'display', 'lat': 44.80054540165129...",27050,55379,US,Shakopee,MN,United States,"[1037 1st Ave E, Shakopee, MN 55379, United St...",NaN,4b5864b8f964a520c35528e3
3,Dunkin',Donut Shop,2425 Rice St,NaN,45.013967,-93.106732,"[{'label': 'display', 'lat': 45.01396688648932...",13595,55113,US,Roseville,MN,United States,"[2425 Rice St, Roseville, MN 55113, United Sta...",NaN,58517cc7739d855b44e53255
4,Glam Doll Donuts,Donut Shop,2605 Nicollet Ave,at E 26th St.,44.955156,-93.277816,"[{'label': 'display', 'lat': 44.95515607246256...",1885,55408,US,Minneapolis,MN,United States,"[2605 Nicollet Ave (at E 26th St.), Minneapoli...",NaN,51254ee3e4b0597635a30975
5,Mel-O-Glaze Bakery,Bakery,4800 28th Ave S,at E Minnehaha Pkwy,44.915864,-93.232306,"[{'label': 'display', 'lat': 44.91586386149392...",6725,55417,US,Minneapolis,MN,United States,"[4800 28th Ave S (at E Minnehaha Pkwy), Minnea...",NaN,451f6c35f964a5209e3a1fe3
6,The Thirsty Whale,Bakery,4149 Fremont Ave N,NaN,45.031281,-93.295872,"[{'label': 'display', 'lat': 45.03128143363127...",7089,55412,US,Minneapolis,MN,United States,"[4149 Fremont Ave N, Minneapolis, MN 55412, Un...",Webber - Camden,5aa4240b4a1cc021996bbdb3
7,Dunkin',Donut Shop,NaN,NaN,44.884108,-93.308363,"[{'label': 'display', 'lat': 44.88410763898639...",10142,55423,US,Minneapolis,MN,United States,"[Minneapolis, MN 55423, United States]",NaN,5beee5d5625a66002c783fba
8,Cardigan Donuts,Donut Shop,40 S 7th St #207,NaN,44.977726,-93.273249,"[{'label': 'display', 'lat': 44.97772595833002...",879,55454,US,Minneapolis,MN,United States,"[40 S 7th St #207, Minneapolis, MN 55454, Unit...",NaN,59144c1a5d891b0a80017201
9,YoYo Donuts & Coffee Bar,Donut Shop,5757 Sanibel Dr,at Shady Oak Road,44.900313,-93.422334,"[{'label': 'display', 'lat': 44.90031269951569...",14451,55343,US,Hopkins,MN,United States,"[5757 Sanibel Dr (at Shady Oak Road), Hopkins,...",NaN,4bc4d71fabf495213ea5c593


In [19]:
dataframe_filtered.name

0                           Dunkin'
1     Angel Food Bakery & Donut Bar
2                  Donut Connection
3                           Dunkin'
4                  Glam Doll Donuts
5                Mel-O-Glaze Bakery
6                 The Thirsty Whale
7                           Dunkin'
8                   Cardigan Donuts
9          YoYo Donuts & Coffee Bar
10                          Dunkin'
11                          Dunkin'
12            Bogart's Doughnut Co.
13                 Glam Doll Donuts
14                       Sleepy V's
15               Puffy Cream Donuts
16                          Dunkin'
17                          Dunkin'
18                          Mucci's
19                          Dunkin'
20                   Dunkin' Donuts
21                          Dunkin'
22        Denny's 5th Avenue Bakery
Name: name, dtype: object

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around Minneapolis

# add the donut shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map